## 仅供交叉验证 前馈神经网络（NNAR）-按总量分类

In [2]:
import pickle
import numpy as np
import random
from utils import *
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
n_input = 11

读取数据

In [2]:
# gene_arr_path = r'../output/gene_editing/es_with_decay.array'
# transplant_arr_path = r'../output/transplant/es_with_decay.array'

# gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
# transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 17, 10)
Shape of the transplant array: (5141, 17, 10)


### 截断数据
2019年为无效数据

In [3]:
# gene_arr = gene_arr[:, :-1, :]
# transplant_arr = transplant_arr[:, :-1, :]

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


### 规范数据并获取5折交叉检验所需的训练集和验证集

In [4]:
# scaler, data = scale_data(transplant_arr, 'standard')

# # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
# X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2],transplant_arr[:, n_input, -1]
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 按总量切分数据

In [5]:
def split_data_by_es(data, targets):
    total_es = np.sum(data[:, :11, -2], axis=1)
    sorted_index = np.argsort(total_es)
    group_size = len(total_es) // 3
    
    data1, target1 = data[sorted_index[:group_size]], targets[sorted_index[:group_size]]
    data2, target2 = data[sorted_index[group_size:2*group_size]], targets[sorted_index[group_size:2*group_size]]
    data3, target3 = data[sorted_index[2*group_size:]], targets[sorted_index[2*group_size:]]
    
    return data1, target1, data2, target2, data3, target3

### 构建模型

In [6]:
def root_mean_squared_error(y_true, y_pred):
        return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true), axis=-1)) 

# def build_direct_dnn_model():
#     model = keras.models.Sequential()
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dense(256, activation='relu'))
#     model.add(Dense(5))
    
#     optimizer=keras.optimizers.Adam(learning_rate=1e-4)
#     model.compile(loss=root_mean_squared_error, optimizer=optimizer)
#     return model
def build_direct_dnn_model(n_layers=2, n_units=256):
    model = keras.models.Sequential()
    model.add(Flatten())
    for i in range(n_layers):
        model.add(Dense(n_units, activation='relu'))
    model.add(Dense(5))
    
    optimizer=keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(loss=root_mean_squared_error, optimizer=optimizer)
    return model

### 进行训练和评估
使用EarlyStopping和Checkpoint做训练停止方式

In [7]:
def cross_validation(X, y, y_cat, kfold, scaler, n_layers, n_units):
    overall_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }

    annual_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }
    
    for train, test in kfold.split(X, y_cat):
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        models = []
        
        # 按总量划分数据集
        X_train1, y_train1, X_train2, y_train2, X_train3, y_train3 = split_data_by_es(X_train, y_train)
        train_xs = [X_train1, X_train2, X_train3]
        train_ys = [y_train1, y_train2, y_train3]
        
        X_test1, y_test1, X_test2, y_test2, X_test3, y_test3 = split_data_by_es(X_test, y_test)
        test_xs = [X_test1, X_test2, X_test3]
        test_ys = [y_test1, y_test2, y_test3]
        i_s = [1, 2, 3]
        
        # 训练
        for i in range(len(i_s)):
            model = build_direct_dnn_model(n_layers, n_units)
            history = model.fit(train_xs[i], train_ys[i], epochs=300, batch_size=16, verbose=1, validation_data=(test_xs[i], test_ys[i]),
                           callbacks=[
                               EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto', restore_best_weights=True)
                           ])
            models.append(model)
        
        # 预测
        y_test = []
        y_pred = []
        for i in range(len(i_s)):
            y_test.append(test_ys[i])
            y_pred.append(models[i].predict(test_xs[i]).reshape(test_ys[i].shape))
        
        y_test = np.concatenate(y_test)
        y_pred = np.concatenate(y_pred)

        metrics = ['mae', 'rmse','ndcg', 'mape', 'r2', 'pearson', 'acc']
        for m in metrics:
            overall, annual = eval_model(m, y_test, y_pred, scaler)
            overall_metrics[m].append(overall)
            annual_metrics[m].append(annual)
    
    return overall_metrics, annual_metrics

In [8]:
def full_pipeline():
    gene_arr_path = r'../output/gene_editing/es_with_decay.array'
    transplant_arr_path = r'../output/transplant/es_with_decay.array'

    gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
    transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))
    
    gene_arr = gene_arr[:, :-1, :]
    transplant_arr = transplant_arr[:, :-1, :]

    print('Shape of the gene_editing array:',gene_arr.shape)
    print('Shape of the transplant array:',transplant_arr.shape)
    
    metrics = {
        'gene':{
            'overall':{},
            'annual':{}
        },
        'transplant':{
            'overall':{},
            'annual':{}
        }
    }
    
    for name, dataset in zip(['gene', 'transplant'], [gene_arr, transplant_arr]):
        scaler, data = scale_data(dataset, 'standard')

        # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
        X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2], dataset[:, n_input, -1]
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        if name == 'gene':
            overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler, 3, 128)
        elif name == 'transplant':
            overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler, 5, 256)

        
        for metric, value in overall_metrics.items():
            metrics[name]['overall'][metric] = np.mean(value)
        
        for metric, value in annual_metrics.items():
            metrics[name]['annual'][metric] = np.mean(np.array(value), axis=0)
    
    pickle.dump(metrics, open('mlp_metrics.dict', 'wb'))
    
    return metrics

In [9]:
metrics = full_pipeline()

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)
Train on 704 samples, validate on 177 samples
Epoch 1/300
704/704 [==============================] - 1s 2ms/sample - loss: 0.8943 - val_loss: 0.6881
Epoch 2/300
704/704 [==============================] - 0s 207us/sample - loss: 0.7863 - val_loss: 0.6646
Epoch 3/300
704/704 [==============================] - 0s 198us/sample - loss: 0.7767 - val_loss: 0.6652
Epoch 4/300
704/704 [==============================] - 0s 208us/sample - loss: 0.7769 - val_loss: 0.6652
Epoch 5/300
704/704 [==============================] - 0s 198us/sample - loss: 0.7770 - val_loss: 0.6649
Epoch 6/300
704/704 [==============================] - 0s 207us/sample - loss: 0.7768 - val_loss: 0.6649
Epoch 7/300
704/704 [==============================] - 0s 205us/sample - loss: 0.7770 - val_loss: 0.6649
Epoch 8/300
704/704 [==============================] - 0s 197us/sample - loss: 0.7772 - val_loss: 0.6653
Epoch 9/300
704/704 [=

704/704 [==============================] - 0s 205us/sample - loss: 0.7585 - val_loss: 0.7483
Epoch 3/300
704/704 [==============================] - 0s 214us/sample - loss: 0.7508 - val_loss: 0.7486
Epoch 4/300
704/704 [==============================] - 0s 227us/sample - loss: 0.7511 - val_loss: 0.7484
Epoch 5/300
704/704 [==============================] - 0s 225us/sample - loss: 0.7513 - val_loss: 0.7486
Epoch 6/300
704/704 [==============================] - 0s 217us/sample - loss: 0.7513 - val_loss: 0.7473
Epoch 7/300
704/704 [==============================] - 0s 263us/sample - loss: 0.7513 - val_loss: 0.7483
Epoch 8/300
704/704 [==============================] - 0s 229us/sample - loss: 0.7509 - val_loss: 0.7483
Epoch 9/300
704/704 [==============================] - 0s 225us/sample - loss: 0.7510 - val_loss: 0.7484
Epoch 10/300
704/704 [==============================] - 0s 210us/sample - loss: 0.7513 - val_loss: 0.7489
Epoch 11/300
704/704 [==============================] - 0s 214us/s

Epoch 47/300
705/705 [==============================] - 0s 260us/sample - loss: 0.9994 - val_loss: 1.1780
Epoch 48/300
705/705 [==============================] - 0s 265us/sample - loss: 1.0007 - val_loss: 1.1767
Epoch 49/300
705/705 [==============================] - 0s 293us/sample - loss: 0.9964 - val_loss: 1.1774
Epoch 50/300
705/705 [==============================] - 0s 267us/sample - loss: 0.9950 - val_loss: 1.1775
Epoch 51/300
705/705 [==============================] - 0s 272us/sample - loss: 0.9926 - val_loss: 1.1788
Epoch 52/300
705/705 [==============================] - 0s 266us/sample - loss: 0.9903 - val_loss: 1.1777
Epoch 53/300
705/705 [==============================] - 0s 259us/sample - loss: 0.9884 - val_loss: 1.1791
Epoch 54/300
705/705 [==============================] - 0s 266us/sample - loss: 0.9869 - val_loss: 1.1772
Epoch 55/300
705/705 [==============================] - 0s 263us/sample - loss: 0.9851 - val_loss: 1.1815
Epoch 56/300
705/705 [========================

705/705 [==============================] - 0s 257us/sample - loss: 1.1380 - val_loss: 1.1294
Epoch 6/300
705/705 [==============================] - 0s 263us/sample - loss: 1.1313 - val_loss: 1.1259
Epoch 7/300
705/705 [==============================] - 0s 272us/sample - loss: 1.1243 - val_loss: 1.1217
Epoch 8/300
705/705 [==============================] - 0s 257us/sample - loss: 1.1184 - val_loss: 1.1228
Epoch 9/300
705/705 [==============================] - 0s 263us/sample - loss: 1.1140 - val_loss: 1.1212
Epoch 10/300
705/705 [==============================] - 0s 267us/sample - loss: 1.1095 - val_loss: 1.1226
Epoch 11/300
705/705 [==============================] - 0s 267us/sample - loss: 1.1053 - val_loss: 1.1190
Epoch 12/300
705/705 [==============================] - 0s 250us/sample - loss: 1.1006 - val_loss: 1.1199
Epoch 13/300
705/705 [==============================] - 0s 257us/sample - loss: 1.0969 - val_loss: 1.1211
Epoch 14/300
705/705 [==============================] - 0s 256u

706/706 [==============================] - 0s 333us/sample - loss: 1.1289 - val_loss: 1.1475
Epoch 9/300
706/706 [==============================] - 0s 283us/sample - loss: 1.1232 - val_loss: 1.1520
Epoch 10/300
706/706 [==============================] - 0s 283us/sample - loss: 1.1176 - val_loss: 1.1493
Epoch 11/300
706/706 [==============================] - 0s 275us/sample - loss: 1.1151 - val_loss: 1.1503
Epoch 12/300
706/706 [==============================] - 0s 277us/sample - loss: 1.1113 - val_loss: 1.1557
Epoch 13/300
706/706 [==============================] - 0s 288us/sample - loss: 1.1071 - val_loss: 1.1606
Epoch 14/300
706/706 [==============================] - 0s 283us/sample - loss: 1.1041 - val_loss: 1.1590
Epoch 15/300
706/706 [==============================] - 0s 288us/sample - loss: 1.1012 - val_loss: 1.1655
Train on 705 samples, validate on 175 samples
Epoch 1/300
705/705 [==============================] - 1s 2ms/sample - loss: 0.8756 - val_loss: 0.7078
Epoch 2/300
705/7

706/706 [==============================] - 0s 275us/sample - loss: 1.0761 - val_loss: 1.0772
Epoch 27/300
706/706 [==============================] - 0s 274us/sample - loss: 1.0730 - val_loss: 1.0779
Epoch 28/300
706/706 [==============================] - 0s 274us/sample - loss: 1.0708 - val_loss: 1.0771
Epoch 29/300
706/706 [==============================] - 0s 267us/sample - loss: 1.0673 - val_loss: 1.0766
Epoch 30/300
706/706 [==============================] - 0s 264us/sample - loss: 1.0648 - val_loss: 1.0764
Epoch 31/300
706/706 [==============================] - 0s 259us/sample - loss: 1.0628 - val_loss: 1.0768
Epoch 32/300
706/706 [==============================] - 0s 274us/sample - loss: 1.0607 - val_loss: 1.0763
Epoch 33/300
706/706 [==============================] - 0s 302us/sample - loss: 1.0576 - val_loss: 1.0765
Epoch 34/300
706/706 [==============================] - 0s 294us/sample - loss: 1.0545 - val_loss: 1.0763
Epoch 35/300
706/706 [==============================] - 0s 

1370/1370 [==============================] - 0s 321us/sample - loss: 0.3964 - val_loss: 0.4081
Epoch 8/300
1370/1370 [==============================] - 0s 317us/sample - loss: 0.3879 - val_loss: 0.4118
Epoch 9/300
1370/1370 [==============================] - 0s 317us/sample - loss: 0.3804 - val_loss: 0.4084
Epoch 10/300
1370/1370 [==============================] - 0s 312us/sample - loss: 0.3726 - val_loss: 0.4085
Epoch 11/300
1370/1370 [==============================] - 0s 324us/sample - loss: 0.3618 - val_loss: 0.4135
Epoch 12/300
1370/1370 [==============================] - 0s 363us/sample - loss: 0.3538 - val_loss: 0.4106
Epoch 13/300
1370/1370 [==============================] - 0s 361us/sample - loss: 0.3436 - val_loss: 0.4187
Epoch 14/300
1370/1370 [==============================] - 0s 331us/sample - loss: 0.3367 - val_loss: 0.4123
Epoch 15/300
1370/1370 [==============================] - 0s 324us/sample - loss: 0.3263 - val_loss: 0.4189
Train on 1371 samples, validate on 344 samp

1370/1370 [==============================] - 2s 1ms/sample - loss: 0.4157 - val_loss: 0.4015
Epoch 2/300
1370/1370 [==============================] - 0s 320us/sample - loss: 0.3947 - val_loss: 0.3976
Epoch 3/300
1370/1370 [==============================] - 0s 338us/sample - loss: 0.3907 - val_loss: 0.3949
Epoch 4/300
1370/1370 [==============================] - 0s 315us/sample - loss: 0.3862 - val_loss: 0.3936
Epoch 5/300
1370/1370 [==============================] - 0s 341us/sample - loss: 0.3839 - val_loss: 0.3934
Epoch 6/300
1370/1370 [==============================] - 0s 346us/sample - loss: 0.3802 - val_loss: 0.3949
Epoch 7/300
1370/1370 [==============================] - 0s 357us/sample - loss: 0.3793 - val_loss: 0.3977
Epoch 8/300
1370/1370 [==============================] - 0s 334us/sample - loss: 0.3766 - val_loss: 0.3921
Epoch 9/300
1370/1370 [==============================] - 0s 313us/sample - loss: 0.3732 - val_loss: 0.3979
Epoch 10/300
1370/1370 [===========================

1372/1372 [==============================] - 0s 358us/sample - loss: 0.9858 - val_loss: 0.9713
Epoch 3/300
1372/1372 [==============================] - 0s 326us/sample - loss: 0.9625 - val_loss: 0.9716
Epoch 4/300
1372/1372 [==============================] - 0s 355us/sample - loss: 0.9447 - val_loss: 0.9669
Epoch 5/300
1372/1372 [==============================] - 0s 342us/sample - loss: 0.9315 - val_loss: 0.9597
Epoch 6/300
1372/1372 [==============================] - 0s 343us/sample - loss: 0.9191 - val_loss: 0.9585
Epoch 7/300
1372/1372 [==============================] - 1s 370us/sample - loss: 0.9082 - val_loss: 0.9608
Epoch 8/300
1372/1372 [==============================] - 0s 327us/sample - loss: 0.8951 - val_loss: 0.9573
Epoch 9/300
1372/1372 [==============================] - 0s 326us/sample - loss: 0.8832 - val_loss: 0.9576
Epoch 10/300
1372/1372 [==============================] - 1s 454us/sample - loss: 0.8665 - val_loss: 0.9643
Epoch 11/300
1372/1372 [========================

In [10]:
metrics

{'gene': {'overall': {'mae': 0.729087191762223,
   'rmse': 1.2839040305460443,
   'ndcg': 0.45843922112693863,
   'mape': 4.479670800712651,
   'r2': 0.24605467970300313,
   'pearson': 0.5256146778643038,
   'acc': 0.3389295492412082},
  'annual': {'mae': array([0.40826308, 0.57645428, 0.75528763, 0.88053546, 1.02489551]),
   'rmse': array([0.74224636, 0.9580917 , 1.31857998, 1.48048021, 1.66244252]),
   'ndcg': array([0.48617829, 0.43226526, 0.16332732, 0.24416067, 0.16687128]),
   'mape': array([3.20163363, 2.79357391, 3.35044904, 6.30414468, 6.74855274]),
   'r2': array([0.44522414, 0.29500761, 0.18427405, 0.06208507, 0.02513317]),
   'pearson': array([0.68097563, 0.5689953 , 0.47024685, 0.33002525, 0.27093228]),
   'acc': array([0.60837437, 0.19376762, 0.23379079, 0.34087837, 0.31783659])}},
 'transplant': {'overall': {'mae': 0.7670352568953716,
   'rmse': 1.267691327415828,
   'ndcg': 0.48531922717613984,
   'mape': 3.808742305883883,
   'r2': 0.4261475457845016,
   'pearson': 0.6

In [8]:
metrics

{'gene': {'overall': {'mae': 0.7323172166659129,
   'rmse': 1.296150959780827,
   'ndcg': 0.39375426995433405,
   'mape': 4.407860713666826,
   'r2': 0.23216118467795743,
   'pearson': 0.5154964653492857,
   'acc': 0.34331252699477943},
  'annual': {'mae': array([0.41667306, 0.57881098, 0.75487069, 0.88225787, 1.02897349]),
   'rmse': array([0.75489929, 0.98624051, 1.31984687, 1.49443502, 1.67494569]),
   'ndcg': array([0.51731997, 0.21897933, 0.15831747, 0.16538815, 0.11366666]),
   'mape': array([3.19242965, 2.7242381 , 3.31875722, 6.27535841, 6.52852019]),
   'r2': array([0.42758378, 0.25620273, 0.18235494, 0.04360206, 0.01077944]),
   'pearson': array([0.66936208, 0.53171588, 0.47137955, 0.31838994, 0.25930228]),
   'acc': array([0.60385537, 0.21832896, 0.24209849, 0.33745923, 0.31482059])}},
 'transplant': {'overall': {'mae': 0.7708396953327952,
   'rmse': 1.2728291969176666,
   'ndcg': 0.49547971873051616,
   'mape': 3.814376280019359,
   'r2': 0.42156821867225547,
   'pearson': 